<a href="https://colab.research.google.com/github/petermankowski510/ALS_LILIGO_Tracking/blob/main/Meeting_Minutes_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 14.5 MB/s eta 0:00:00


In [2]:
pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.0
    Uninstalling openai-1.61.0:
      Successfully uninstalled openai-1.61.0


In [3]:
!pip -q install langchain-core langchain-community langgraph langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [57]:
!pip install keyboard
!apt-get install -y portaudio19-dev libportaudio2 libportaudiocpp0
!pip install pyaudio
!apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1.1).
libportaudiocpp0 is already the newest version (19.6.0-1.1).
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [58]:
import speech_recognition as sr
import openai
import datetime

In [59]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_AI_KEY')

In [60]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [62]:
def transcribe_audio(filename):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return "Audio not clear. Could not transcribe."
    except sr.RequestError:
        return "Error with the speech recognition service."

def summarize_text(text):
    # Use predict_messages instead of create for chat models in LangChain
    from langchain.schema import (
        AIMessage,
        HumanMessage,
        SystemMessage
    )
    messages = [HumanMessage(content=f"You are am AI assitant, taking meeting minutes. Summarize the following text, adding a section describing sentiment of the speaker: {text}")]
    response = llm.predict_messages(messages)
    return response.content  # Access the content of the AI message

In [63]:
def save_meeting_notes(summary):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"meeting_notes_{timestamp}.txt"
    with open(filename, "w") as file:
        file.write(summary)
    print(f"Meeting notes saved as {filename}")

In [64]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import scipy.io.wavfile as wavfile

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=20): # Record for 20 seconds by default
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.webm', 'wb') as f:
    f.write(b)

  # Convert to WAV (if needed)
  !ffmpeg -i audio.webm audio.wav  # Make sure FFmpeg is installed

  return 'audio.wav'

print("Recording will start now and run for 5 seconds.")
audio_file = record()
print(f"Audio recording saved as: {audio_file}")


Recording will start now and run for 5 seconds.


<IPython.core.display.Javascript object>

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [65]:
# prompt: ModuleNotFoundError: No module named 'pyaudio'

import speech_recognition as sr
import openai
import datetime
import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# Install pyaudio if not installed
!apt-get install -y portaudio19-dev
!pip install pyaudio

!pip install SpeechRecognition
!pip install --upgrade openai
!pip -q install langchain-core langchain-community langgraph langchain-openai

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_AI_KEY')

llm = ChatOpenAI(model="gpt-4o")

def transcribe_audio(filename):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return "Audio not clear. Could not transcribe."
    except sr.RequestError:
        return "Error with the speech recognition service."

def summarize_text(text):
    # Use predict_messages instead of create for chat models in LangChain
    messages = [HumanMessage(content=f"You are am AI assitant, taking meeting minutes. Summarize the following text, adding a section describing sentiment of the speaker: {text}")]
    response = llm.predict_messages(messages)
    return response.content  # Access the content of the AI message

def save_meeting_notes(summary):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"meeting_notes_{timestamp}.txt"
    with open(filename, "w") as file:
        file.write(summary)
    print(f"Meeting notes saved as {filename}")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [66]:
def main():
    audio_file = "/content/audio.wav"
    transcript = transcribe_audio(audio_file)
    print("Transcript:", transcript)

    summary = summarize_text(transcript)
    print("Summary:", summary)

    save_meeting_notes(summary)

if __name__ == "__main__":
    main()


Transcript: okay let's review our q1 and financial reports with Jessica Mike and Jack we made profit of 25 million in Canada the best Province was Alberta with sales increase of
Summary: **Meeting Summary**

**Attendees:** 
- Jessica
- Mike
- Jack

**Topics Discussed:**
1. **Q1 and Financial Reports Review**
   - Overall profit: $25 million in Canada.
   - Top-performing province: Alberta, noted for a significant increase in sales.

**Sentiment Analysis:**
The sentiment of the speaker appears to be positive and optimistic, reflecting satisfaction with the financial results and Alberta’s sales performance. The emphasis on the profit and specific mention of Alberta's success suggests a focus on achievements and potential growth opportunities.
Meeting notes saved as meeting_notes_2025-02-07_05-24-12.txt
